In [1]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
from rasterio.warp import transform
import xarray as xr
import richdem as rd
import geopandas as gpd
# Initialize the Earth Engine module.
ee.Initialize()

In [15]:
#########################################################################
############################ USER INPUTS ################################
#########################################################################
# PATHS

# DOMAIN
# choose the modeling domain
domain = 'YUKO'

# path to temporary folder to store tif files from gee
TIFpath = domain + '_GEE_Downloads/'

# path to directory where you want your output .tif and .asc files
dataPath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/'
#########################################################################

# Download DEM and LC data from GEE

In [8]:
# # Download dem and veg data function
# def get_topoveg(domain, OUTpath):
    
#     #path to CSO domains
#     domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
#     domains = domains_resp.json()

#     '''
#     // These are the min and max corners of your domain in Lat, Long
#     // Western Wyoming
#     // Input the minimum lat, lower left corner
#     '''
#     minLat = domains[domain]['Bbox']['latmin']
#     #// Input the minimum long, lower left corner
#     minLong = domains[domain]['Bbox']['lonmin']
#     #// Input the max lat, upper right corner
#     maxLat = domains[domain]['Bbox']['latmax']
#     #// Input the max Long, upper right corner
#     maxLong = domains[domain]['Bbox']['lonmax']


#     # This resolution for the NLCD and DEM outputs for the SnowModel domain in meters
#     sm_resolution = int(domains[domain]['cellsize'])

#     '''// Resolution for the PRISM output. This shoud change by Latitude of the domain
#     // because the PRISM product spatial resolution is 2.5 minutes, which equals 150 arc seconds.
#     // You can use this arc-second calculator to estimate the correct value for the PRISM resolution by latitude
#     // https://opendem.info/arc2meters.html
#     // This is one arc-second in meters for 43 degrees N Latitude'''
#     one_arcsecond = 22.57
#     PRISM_resolution = one_arcsecond * 150

#     '''// Define the final output projection using EPSG codes'''
#     epsg_code = domains[domain]['mod_proj']

#     my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
    
#         #################### adjust for AK ##########################
#     #need to specifiy different GEE products for above 60 deg lat
#     #if maxLat>60:
    
#     # Use the GMTED 2010 elevation data for the DEM
#     GMTED = ee.Image('USGS/GMTED2010').select('be75');
#     filename = os.path.join(OUTpath, domain+'_dem.tif')
#     geemap.ee_export_image(GMTED, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

    
#     # download NLCD data for AK
#     NLCD = ee.Image('USGS/NLCD/NLCD2011_AK');
#     lc = NLCD.select('landcover');
#     sm_class = ee.Image(-9999) \
#         .where(lc.eq(11), 19) \
#         .where(lc.eq(12), 20) \
#         .where(lc.eq(21), 21) \
#         .where(lc.eq(22), 21) \
#         .where(lc.eq(23), 21) \
#         .where(lc.eq(24), 21) \
#         .where(lc.eq(31), 18) \
#         .where(lc.eq(41), 2) \
#         .where(lc.eq(42), 1) \
#         .where(lc.eq(43), 6) \
#         .where(lc.eq(51), 6) \
#         .where(lc.eq(52), 6) \
#         .where(lc.eq(71), 12) \
#         .where(lc.eq(72), 12) \
#         .where(lc.eq(73), 12) \
#         .where(lc.eq(74), 12) \
#         .where(lc.eq(81), 23) \
#         .where(lc.eq(82), 22) \
#         .where(lc.eq(90), 9) \
#         .where(lc.eq(95), 9)

#     # mask out invalid values
#     landMask = sm_class.select('constant').gt(0)
#     # select only valid parts of the image
#     NLCDmasked = sm_class.select('constant').updateMask(landMask).cast({'constant': ee.PixelType('int', 1, 24)})
    
#     filename = os.path.join(OUTpath, domain+'_veg.tif');
#     geemap.ee_export_image(sm_class, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

In [9]:
# # landcover data 
# def LC2SM(INfile,OUTpath):
#     da = xr.open_rasterio(INfile)
#     data = np.squeeze(da.values)

#     #ascii header 
#     head = "ncols\t"+str(da.shape[2])+"\n" \
#     "nrows\t"+str(da.shape[1])+"\n" \
#     "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
#     "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
#     "cellsize\t"+str(int(da.res[0]))+"\n" \
#     "NODATA_value\t-9999"
    
#     #reassign lc from NLCD to SM classes
#     DIR=np.empty([da.shape[1],da.shape[2]])
#     DIR[data == 11 ]=19
#     DIR[data == 12 ]=20
#     DIR[data == 21 ]=21
#     DIR[data == 22 ]=21
#     DIR[data == 23 ]=21
#     DIR[data == 24 ]=21
#     DIR[data == 31 ]=18
#     DIR[data == 41 ]=2
#     DIR[data == 42 ]=1
#     DIR[data == 43 ]=6
#     DIR[data == 51 ]=6
#     DIR[data == 52 ]=6
#     DIR[data == 71 ]=12
#     DIR[data == 72 ]=12
#     DIR[data == 73 ]=12
#     DIR[data == 74 ]=12
#     DIR[data == 81 ]=23
#     DIR[data == 82 ]=22
#     DIR[data == 90 ]=9
#     DIR[data == 95 ]=9
#     DIR.astype(int)
#     np.savetxt(OUTpath+'NLCD2016_'+domain+'.asc', DIR, fmt='%d', header = head,comments='')

In [10]:
# # Download dem and veg data function combine NLCD and Copernicus
# def get_topoveg(domain, OUTpath):
    
#     #path to CSO domains
#     domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
#     domains = domains_resp.json()

#     # These are the min and max corners of your domain in Lat, Long
#     minLat = domains[domain]['Bbox']['latmin']
#     #// Input the minimum long, lower left corner
#     minLong = domains[domain]['Bbox']['lonmin']
#     #// Input the max lat, upper right corner
#     maxLat = domains[domain]['Bbox']['latmax']
#     #// Input the max Long, upper right corner
#     maxLong = domains[domain]['Bbox']['lonmax']

#     # The resolution for the NLCD and DEM outputs for the SnowModel domain in meters
#     sm_resolution = int(domains[domain]['cellsize'])

#     # Define the final output projection using EPSG codes
#     epsg_code = domains[domain]['mod_proj']

#     my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});
    
#         #################### adjust for AK ##########################
#     #need to specifiy different GEE products for above 60 deg lat
#     #if maxLat>60:
    
#     # Use the GMTED 2010 elevation data for the DEM
#     GMTED = ee.Image('USGS/GMTED2010').select('be75');
#     filename = os.path.join(OUTpath, domain+'_dem.tif')
#     geemap.ee_export_image(GMTED, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

    
#     # download NLCD data for AK
#     NLCD = ee.Image('USGS/NLCD/NLCD2011_AK');
#     lc = NLCD.select('landcover');
#     sm_classAK = ee.Image(-9999) \
#         .where(lc.eq(11), 19) \
#         .where(lc.eq(12), 20) \
#         .where(lc.eq(21), 21) \
#         .where(lc.eq(22), 21) \
#         .where(lc.eq(23), 21) \
#         .where(lc.eq(24), 21) \
#         .where(lc.eq(31), 18) \
#         .where(lc.eq(41), 2) \
#         .where(lc.eq(42), 1) \
#         .where(lc.eq(43), 6) \
#         .where(lc.eq(51), 6) \
#         .where(lc.eq(52), 6) \
#         .where(lc.eq(71), 12) \
#         .where(lc.eq(72), 12) \
#         .where(lc.eq(73), 12) \
#         .where(lc.eq(74), 12) \
#         .where(lc.eq(81), 23) \
#         .where(lc.eq(82), 22) \
#         .where(lc.eq(90), 9) \
#         .where(lc.eq(95), 9)

#     # mask out invalid values
#     landMask = sm_classAK.select('constant').gt(0)
#     # select only valid parts of the image
#     NLCDmasked = sm_classAK.select('constant').updateMask(landMask).cast({'constant': ee.PixelType('int', 1, 24)})
    
#     # Use the Copernicus land cover product for the veg type
#     copernicus = ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V/Global')
#     lc = copernicus.select('discrete_classification').toBands()
    
#     sm_class = ee.Image(24) \
#         .where(lc.eq(0), 18) \
#         .where(lc.eq(20), 6) \
#         .where(lc.eq(30), 12) \
#         .where(lc.eq(40), 22) \
#         .where(lc.eq(50), 21) \
#         .where(lc.eq(60), 18) \
#         .where(lc.eq(70), 20) \
#         .where(lc.eq(80), 19) \
#         .where(lc.eq(90), 9) \
#         .where(lc.eq(100), 18) \
#         .where(lc.eq(111), 1) \
#         .where(lc.eq(112), 1) \
#         .where(lc.eq(113), 2) \
#         .where(lc.eq(114), 2) \
#         .where(lc.eq(115), 3) \
#         .where(lc.eq(116), 3) \
#         .where(lc.eq(121), 4) \
#         .where(lc.eq(122), 1) \
#         .where(lc.eq(123), 2) \
#         .where(lc.eq(124), 2) \
#         .where(lc.eq(125), 3) \
#         .where(lc.eq(126), 3) \
#         .where(lc.eq(200), 24)
    
#     #combine images
#     combined_img = ee.ImageCollection([sm_class,NLCDmasked]).mosaic()
    
    
#     # export lc image to tif
#     filename = os.path.join(OUTpath, domain+'_veg.tif');
#     geemap.ee_export_image(combined_img, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

In [11]:
# Download dem and veg data function Copernicus only
def get_topoveg(domain, OUTpath):
    
    #path to CSO domains
    domains_resp = requests.get("https://raw.githubusercontent.com/NPRB/02_preprocess_python/main/NPRB_domains.json")
    domains = domains_resp.json()

    # These are the min and max corners of your domain in Lat, Long
    minLat = domains[domain]['Bbox']['latmin']
    #// Input the minimum long, lower left corner
    minLong = domains[domain]['Bbox']['lonmin']
    #// Input the max lat, upper right corner
    maxLat = domains[domain]['Bbox']['latmax']
    #// Input the max Long, upper right corner
    maxLong = domains[domain]['Bbox']['lonmax']

    # The resolution for the NLCD and DEM outputs for the SnowModel domain in meters
    sm_resolution = int(domains[domain]['cellsize'])

    # Define the final output projection using EPSG codes
    epsg_code = domains[domain]['mod_proj']

    my_domain = ee.Geometry.Rectangle(**{'coords':[minLong,minLat,maxLong,maxLat],'proj': 'EPSG:4326','geodesic':True,});

    # Use the GMTED 2010 elevation data for the DEM
    GMTED = ee.Image('USGS/GMTED2010').select('be75');
    # GMTED = ee.Image('NASA/ASTER_GED/AG100_003');
    
    filename = os.path.join(OUTpath, domain+'_dem.tif')
    geemap.ee_export_image(GMTED, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

    # Use the Copernicus land cover product for the veg type
    copernicus = ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V/Global')
    lc = copernicus.select('discrete_classification').toBands()
    
    sm_class = ee.Image(24) \
        .where(lc.eq(0), 18) \
        .where(lc.eq(20), 6) \
        .where(lc.eq(30), 12) \
        .where(lc.eq(40), 22) \
        .where(lc.eq(50), 21) \
        .where(lc.eq(60), 18) \
        .where(lc.eq(70), 20) \
        .where(lc.eq(80), 19) \
        .where(lc.eq(90), 9) \
        .where(lc.eq(100), 18) \
        .where(lc.eq(111), 1) \
        .where(lc.eq(112), 1) \
        .where(lc.eq(113), 2) \
        .where(lc.eq(114), 2) \
        .where(lc.eq(115), 3) \
        .where(lc.eq(116), 3) \
        .where(lc.eq(121), 4) \
        .where(lc.eq(122), 1) \
        .where(lc.eq(123), 2) \
        .where(lc.eq(124), 2) \
        .where(lc.eq(125), 3) \
        .where(lc.eq(126), 3) \
        .where(lc.eq(200), 24)
    
    
    # export lc image to tif
    filename = os.path.join(OUTpath, domain+'_veg.tif');
    geemap.ee_export_image(sm_class, filename=filename, scale=sm_resolution, region=my_domain, crs = epsg_code);

In [16]:
# execute GEE function
get_topoveg(domain, dataPath)

Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/YUKO/YUKO_dem.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/YUKO/YUKO_veg.tif


In [17]:
# visualize tif with shapefile
# this allows you to ensure that the bounding box covers the full domain
DEMfile = dataPath+domain+'_dem.tif'
dem = xr.open_rasterio(DEMfile)
shp = gpd.read_file('/nfs/attic/dfh/2020_NPRB/data/gis/'+domain+'/'+domain+'.json')
shpproj = shp.to_crs(dem.crs)
#plot
fig,ax = plt.subplots()
dem.plot(ax=ax)
shpproj.plot(ax=ax,color='c')

/nfs/attic/dfh/miniconda/envs/ee/lib/python3.9/site-packages/pyproj/crs/crs.py:292: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


## DEM with LC
### Edit landcover and dem to prevent negative discharge values

1. set the elevation of any dem pixels corresponding to ocean landcover, to 0

2. Set and landcover pixels that correspond to zero elevation, to ocean

3. fill in any depressions

In [20]:
def DEM_LC2SM_fill(DEM_tiffpath,LC_tiffpath,OUTpath,fill=True):
    # open dem .tif
    dem = xr.open_rasterio(DEM_tiffpath)
    # open lc .tif
    lc = xr.open_rasterio(LC_tiffpath)
    #change all cells in the dem that correspond to the ocean lc type to 0 elevation
    dems=dem.where(lc!=24,0)
    #change all cells in the lc that correspond to 0 elevation, to 24
    lcs=lc.where(dems!=0,24)
    #ascii header 
    head = "ncols\t"+str(dem.shape[2])+"\n" \
        "nrows\t"+str(dem.shape[1])+"\n" \
        "xllcorner\t"+str(int(min(dem.x.values)-dem.res[0]/2))+"\n" \
        "yllcorner\t"+str(int(min(dem.y.values)-dem.res[0]/2))+"\n" \
        "cellsize\t"+str(int(dem.res[0]))+"\n" \
        "NODATA_value\t-9999" 

    if fill == True:
        print('generating filled dem')
        #make richdem array
        rda = rd.rdarray(np.squeeze(dems.values), no_data=0)
        #fill depressions in DEM to level of lowest outlet or spill-point
        dem_filled = rd.FillDepressions(rda, in_place=False)
        #save filled dem
        np.savetxt(OUTpath+domain+'_dem.asc', dem_filled, fmt='%d', header = head,comments='')
    elif fill == False:
        print('dem not filled')
        #save dem
        np.savetxt(OUTpath+domain+'_dem.asc', np.squeeze(dems.values), fmt='%d', header = head,comments='') 

    #save lc
    np.savetxt(OUTpath+domain+'_veg.asc', np.squeeze(lcs.values), fmt='%d', header = head,comments='')
    

# Lat long grids

In [21]:
def LTLN2SM(INfile,OUTpath):
    da = xr.open_rasterio(INfile)

    # Compute the lon/lat coordinates with rasterio.warp.transform
    ny, nx = len(da.y), len(da.x)
    x, y = np.meshgrid(da.x, da.y)

    # Rasterio works with 1D arrays
    lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
                         x.flatten(), y.flatten())
    lon = np.asarray(lon).reshape((ny, nx))
    lat = np.asarray(lat).reshape((ny, nx))

    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"
    np.savetxt(OUTpath+domain+'_grid_lat.asc', lat, fmt='%2.5f', header = head,comments='')
    np.savetxt(OUTpath+domain+'_grid_lon.asc', lon, fmt='%4.5f', header = head,comments='')
    
    inF = OUTpath+domain+'_grid_lat.asc'
    outF = OUTpath+domain+'_grid_lat.tif'
    !gdal_translate -of "GTiff" $inF $outF

    inF = OUTpath+domain+'_grid_lon.asc'
    outF = OUTpath+domain+'_grid_lon.tif'
    !gdal_translate -of "GTiff" $inF $outF

# Execute functions - 1 domain

In [8]:
# path to directory where you want your output .tif and .asc files
dataPath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/'

# execute GEE function
get_topoveg(domain, dataPath)

# generate topoveg SM input files
DEM_tiffpath = dataPath+domain+'_dem.tif'
LC_tiffpath = dataPath+domain+'_veg.tif'
DEM_LC2SM_fill(DEM_tiffpath,LC_tiffpath,dataPath)

#generate lat lon grids
LTLN2SM(DEM_tiffpath,dataPath)

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Input file size is 1971, 1086
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 1971, 1086
0...10...20...30...40...50...60...70...80...90...100 - done.


# Execute functions - multiple domains

In [22]:
# AK domains
domain_list = ['BRIS','CHUK','YUKO']
# domain_list = ['BEAU','BRIS','CHUK','JUNE','KLAW','SITK','TAKU','YUKO']

for domain in domain_list:
    print(domain)
    # path to directory where you want your output .tif and .asc files
    dataPath = '/nfs/attic/dfh/2020_NPRB/data/SMinputs/'+domain+'/'

    # execute GEE function
    get_topoveg(domain, dataPath)
    
    # generate topoveg SM input files
    DEM_tiffpath = dataPath+domain+'_dem.tif'
    LC_tiffpath = dataPath+domain+'_veg.tif'
    DEM_LC2SM_fill(DEM_tiffpath,LC_tiffpath,dataPath)
    
    #generate lat lon grids
    LTLN2SM(DEM_tiffpath,dataPath)

BRIS
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/BRIS/BRIS_dem.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/BRIS/BRIS_veg.tif
generating filled dem
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Input file size is 949, 1228
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 949, 1228
0...10...20...30...40...50...60...70...80...90...100 - done.
CHUK
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/CHUK/CHUK_dem.tif
Generating URL ...
Please wait ...
Data downloaded to /nfs/attic/dfh/2020_NPRB/data/SMinputs/CHUK/CHUK_veg.tif
generating filled dem
Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)
Input file size is 763, 798
0...10...20...30...40...50...60...70...80...90...100 - done.

# DEM - old

In [5]:
# dem fill depressions
def DEM2SM_fill(INfile, OUTpath):
    da = xr.open_rasterio(INfile)
    
    #make richdem array
    rda = rd.rdarray(np.squeeze(da.values), no_data=0)
    #fill depressions in DEM to level of lowest outlet or spill-point
    dem_filled = rd.FillDepressions(rda, in_place=False)
    
    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"    
    
    np.savetxt(OUTpath+domain+'_dem.asc', dem_filled, fmt='%d', header = head,comments='')

In [10]:
# generate topo
INfile = dataPath+domain+'_dem.tif'
DEM2SM_fill(INfile, dataPath)

Warning! No geotransform defined. Choosing a standard one! (Top left cell's top let corner at <0,0>; cells are 1x1.)


In [3]:
# dem 
def DEM2SM(INfile, OUTpath):
    da = xr.open_rasterio(INfile)
    
    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"    
    
    np.savetxt(OUTpath+domain+'_dem.asc', np.squeeze(da.values), fmt='%d', header = head,comments='')

In [4]:
# generate topo
INfile = dataPath+domain+'_dem.tif'
DEM2SM(INfile, dataPath)

In [36]:
# landcover data 
def LC2SM(INfile,OUTpath):
    da = xr.open_rasterio(INfile)

    #ascii header 
    head = "ncols\t"+str(da.shape[2])+"\n" \
    "nrows\t"+str(da.shape[1])+"\n" \
    "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
    "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
    "cellsize\t"+str(int(da.res[0]))+"\n" \
    "NODATA_value\t-9999"

    np.savetxt(OUTpath+domain+'_veg.asc', np.squeeze(da.values), fmt='%d', header = head,comments='')

In [5]:
# # landcover data 
# def LC2SM(INfile,OUTpath):
#     da = xr.open_rasterio(INfile)
#     data = np.squeeze(da.values)

#     #ascii header 
#     head = "ncols\t"+str(da.shape[2])+"\n" \
#     "nrows\t"+str(da.shape[1])+"\n" \
#     "xllcorner\t"+str(int(min(da.x.values)-da.res[0]/2))+"\n" \
#     "yllcorner\t"+str(int(min(da.y.values)-da.res[0]/2))+"\n" \
#     "cellsize\t"+str(int(da.res[0]))+"\n" \
#     "NODATA_value\t-9999"
    
#     #reassign lc from NLCD to SM classes
#     DIR=np.empty([da.shape[1],da.shape[2]])
#     DIR[data == 11 ]=19
#     DIR[data == 12 ]=20
#     DIR[data == 21 ]=21
#     DIR[data == 22 ]=21
#     DIR[data == 23 ]=21
#     DIR[data == 24 ]=21
#     DIR[data == 31 ]=18
#     DIR[data == 41 ]=2
#     DIR[data == 42 ]=1
#     DIR[data == 43 ]=6
#     DIR[data == 51 ]=6
#     DIR[data == 52 ]=6
#     DIR[data == 71 ]=12
#     DIR[data == 72 ]=12
#     DIR[data == 73 ]=12
#     DIR[data == 74 ]=12
#     DIR[data == 81 ]=23
#     DIR[data == 82 ]=22
#     DIR[data == 90 ]=9
#     DIR[data == 95 ]=9
#     DIR.astype(int)
#     np.savetxt(OUTpath+domain+'_veg.asc', DIR, fmt='%d', header = head,comments='')